<a href="https://colab.research.google.com/github/manansharma-00/manan/blob/main/SIH_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip -q install langchain pypdf sentence_transformers chromadb

In [ ]:

from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
loader = DirectoryLoader('./Pdf_files/',glob='./*.pdf',loader_cls = PyPDFLoader)
documents = loader.load()

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
loader2 = PyPDFDirectoryLoader("./Pdf_files/")
documents2 = loader2.load()


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 20,
)
texts = text_splitter.split_documents(documents)
texts2 = text_splitter.split_documents(documents2)
texts[3]

Document(page_content='FOREWORD\nAll India Council for Technical Education (AICTE) was established in 1945 as a National Level Apex Advisory \nBody and as a Statutory body by an Act of Parliament in 1987 for developing and promoting quality technical \neducation in the country in a coordinated and integrated manner. AICTE is in constant efforts to improve the \nquality of technical education in India. \nWe, at AICTE, are committed to promote holistic, qualitative, inclusive and accessible education in the country \nwith an aim to make India a technical hub by 2047. AICTE emphasizes social and emotional learning as \na crucial component of Education in line with  NEP 2020 and has introduced  Universal Human Values as \na mandatory portion in the model curriculum. The core strength to build a technically strong ecosystem in \nthe country lies in the hands of our institutions and future generations, who have a bigger role to play in the \nnation building and development of the country.', 

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import Chroma
persist_directory = "db"

vectordb = Chroma.from_documents(documents = texts,embedding = hf,persist_directory=persist_directory)

In [ ]:
vectordb.persist()
vectordb = None

In [ ]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function = hf)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k":5})


In [ ]:
## Default LLaMA-2 prompt style
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

def get_prompt(instruction, new_system_prompt ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template


In [ ]:
sys_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

instruction = """CONTEXT:/n/n {context}/n

Question: {question}"""
get_prompt(instruction, sys_prompt)

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. \n<</SYS>>\n\nCONTEXT:/n/n {context}/n\n\nQuestion: {question}[/INST]"

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = get_prompt(instruction, sys_prompt)

llama_prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
from langchain.llms import Together

llama2_llm = Together(
    model="togethercomputer/llama-2-70b-chat",
    temperature=0.7,
    max_tokens=1024,
    together_api_key="d8ec7106bd0c268bf4672dba83272b86054fbe849eba82f3f75ceb17e6d57eb0"
)


In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
multiretriever = MultiQueryRetriever.from_llm(
    retriever= retriever, llm=llama2_llm
)

In [ ]:
unique_docs = multiretriever.get_relevant_documents(query="Under what conditions will aicte reject my approval")
len(unique_docs)

KeyboardInterrupt: ignored

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm= llama2_llm, chain_type_kwargs = {"prompt": llama_prompt},chain_type="stuff",retriever= retriever,return_source_documents = True)


In [ ]:
def process_llm_response(llm_response):
  print(llm_response['result'])
  print('\n\nSources:')
  for source in llm_response['source_documents']:
    print("Page number - "+str(source.metadata['page'])+" "+source.metadata['source'])

In [ ]:
process_llm_response(qa_chain("Explain what are TER charges"))

 TER charges stand for Technical Education Regulatory charges. These are charges levied by the AICTE (All India Council for Technical Education) for the regulation and monitoring of technical education institutions in India. The charges are applicable to institutions that are seeking approval or affiliation from AICTE, and are used to cover the costs of the regulator's activities, such as conducting inspections, scrutiny committees, and expert visit committees. The TER charges are not refundable in any case, even if the application is rejected.


Sources:
Page number - 29 Pdf_files/APH Final.pdf
Page number - 29 Pdf_files/APH Final.pdf
Page number - 65 Pdf_files/APH Final.pdf
Page number - 65 Pdf_files/APH Final.pdf
Page number - 54 Pdf_files/APH Final.pdf


In [ ]:
process_llm_response(qa_chain("What are the documents required for opening a new institution?"))